# How to Search for Bluesky Data?

Show [how to search](https://github.com/BCDA-APS/apstools/issues/674) for
Bluesky data from a [databroker](https://blueskyproject.io/databroker) catalog.

The databroker [search and
lookup](https://blueskyproject.io/databroker/tutorials/search-and-lookup.html)
tutorial is a great way to learn how to search for Bluesky data.
This How-To guide continues from the tutorial, using additional support from
[apstools](https://BCDA-APS.github.io/apstools).
Custom queries are expressed using [MongoDB
Query](https://www.mongodb.com/docs/manual/reference/operator/query/).
Additional help with the MongoDB query language, operators, and syntax may be
found online.
The content from
[w3schools](https://www.w3schools.com/python/python_mongodb_query.asp) is both
informative and compact.

## What is a catalog?

A *catalog* is a group of Bluesky measurements from the [databroker](https://blueskyproject.io/databroker).

Bluesky uses the term *run* to describe a single measurement in the catalog.  A *run* consists of several parts.  Each part has its own type of document, as follows:

document | description
--- | ---
start | Initial information about the measurement, including metadata.
descriptor | A description of the data to be collected.
event | The measurement data.
stop | A final summary of the measurement.

Here, we create a Python reference to Bluesky *runs* stored in a `class_2021_03` database.

In [1]:
import databroker

print(f"catalogs available: {list(databroker.catalog)=}")

cat = databroker.catalog["class_2021_03"].v2
cat

catalogs available: list(databroker.catalog)=['bdp2022', 'class_2021_03', '6idb_export', 'apstools_test', 'class_data_examples', 'usaxs_test', 'korts202106', 'training']


class_2021_03:
  args:
    asset_registry_db: mongodb://localhost:27017/class_2021_03-bluesky
    metadatastore_db: mongodb://localhost:27017/class_2021_03-bluesky
    name: class_2021_03
  description: ''
  driver: databroker._drivers.mongo_normalized.BlueskyMongoCatalog
  metadata:
    catalog_dir: /home/prjemian/.local/share/intake/


What does the `cat` object describe?

In [2]:
print(f"{cat.name=}")
print(f"{len(cat)=} measurements (runs) in the catalog")
print(f"{cat.metadata=}")

cat.name='class_2021_03'
len(cat)=40 measurements (runs) in the catalog
cat.metadata={'catalog_dir': '/home/prjemian/.local/share/intake/'}


NOTE: A search on a catalog object returns a new catalog as filtered by the search parameters.

## How to retrieve a run?

Retrieve any run from the catalog using one of three references:

reference | example | type | description
--- | --- | --- | ---
`scan_id` | `cat[192]` | *positive* integer | not necessarily unique, returns most recent
relative | `cat[-1]` | *negative* integer | `-1` is most recent run, `-2` is the run before, ...
`uid` | `cat["abc1234"]` | hexadecimal string | unique, matches the first characters of the start document `uid`

Note: Partial representation of the `uid` is allowed, matching from the start of
the full `uid`.  The given *short* `uid` must include enough characters to make
a unique match in the catalog **and** must include up to the first non-numeric
character to avoid mis-interpretation as an integer.  You will be advised if the
*short* `uid` is not a unique match.

Each of these references retrieve the same run:

In [3]:
print(f"scan_id: {cat[192]=}")
print(f"scan_id: {cat['192']=}")  # treated as an integer by databroker
print(f"relative: {cat[-1]=}")
print(f"relative: {cat['-1']=}")  # treated as an integer by databroker
print(f"short uid: {cat['e3']=}")  # shortest version that works
print(f"short uid: {cat['e3862991-688d']=}")  # must include the hyphen

scan_id: cat[192]=<BlueskyRun uid='e3862991-688d-43dc-8442-d85ccbb3d6c8'>
scan_id: cat['192']=<BlueskyRun uid='e3862991-688d-43dc-8442-d85ccbb3d6c8'>
relative: cat[-1]=<BlueskyRun uid='e3862991-688d-43dc-8442-d85ccbb3d6c8'>
relative: cat['-1']=<BlueskyRun uid='e3862991-688d-43dc-8442-d85ccbb3d6c8'>
short uid: cat['e3']=<BlueskyRun uid='e3862991-688d-43dc-8442-d85ccbb3d6c8'>
short uid: cat['e3862991-688d']=<BlueskyRun uid='e3862991-688d-43dc-8442-d85ccbb3d6c8'>


Show the run has the expected `scan_id`.

In [4]:
print(f"scan_id: {cat[192].metadata['start']['scan_id']=}")
cat[-1]

scan_id: cat[192].metadata['start']['scan_id']=192


BlueskyRun
  uid='e3862991-688d-43dc-8442-d85ccbb3d6c8'
  exit_status='success'
  2021-05-19 15:22:03.070 -- 2021-05-19 15:22:07.087
  Streams:
    * baseline
    * primary


There is one more way to retrieve runs from the catalog, iterate over the catalog to get the full `uid` of each run, then use `cat[uid]` to get the run object.  This technique might be more useful on smaller catalogs.

Here, we break after the first one (to limit output):

In [5]:
for uid in cat:
    print(f"{uid=}  {cat[uid]=}")
    break

uid='e3862991-688d-43dc-8442-d85ccbb3d6c8'  cat[uid]=<BlueskyRun uid='e3862991-688d-43dc-8442-d85ccbb3d6c8'>


## Simple, use `listruns()`

The `apstools.utils.listruns()` function from
[apstools](https://bcda-aps.github.io/apstools) provides a listing of the most
recent runs.  Taking all the default settings, `listruns()` shows (up to) the 20
most recent runs in the catalog.

Here, the first column is an index number (and can be ignored), the remaining
columns are labeled.  Note the `time` column includes both the `yyyy-mm-dd` date
and the `HH:MM:SS` time of day (24-hour time).

In [6]:
from apstools.utils import listruns

listruns()

/home/prjemian/.conda/envs/bluesky_2022_3/lib/python3.9/site-packages/databroker/queries.py:89: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  timezone = lz.zone


catalog: class_2021_03
    scan_id                 time plan_name                   detectors
0       192  2021-05-19 15:22:03  rel_scan  [scaler1, adsimdet, zaxis]
1       191  2021-05-19 15:19:27  rel_scan         [scaler1, adsimdet]
2       190  2021-05-19 15:18:59  rel_scan                   [scaler1]
3       189  2021-05-19 15:18:28  rel_scan      [scaler1, temperature]
4       188  2021-05-19 15:14:56  rel_scan                   [scaler1]
5       187  2021-05-19 15:13:47  rel_scan                   [scaler1]
6        33  2021-04-07 15:55:09      scan                     [noisy]
7        33  2021-03-17 00:32:55     count                  [adsimdet]
8        32  2021-03-17 00:31:44     count               [temperature]
9        31  2021-03-17 00:31:24     count               [temperature]
10       30  2021-03-17 00:31:23     count               [temperature]
11       29  2021-03-17 00:31:23     count               [temperature]
12       28  2021-03-17 00:30:33  rel_scan            

There are many options for `listruns()`, show them using the `listruns?` syntax:

In [7]:
listruns?

Signature:
listruns(
    cat=None,
    keys=None,
    missing='',
    num=20,
    printing='smart',
    reverse=True,
    since=None,
    sortby='time',
    tablefmt='dataframe',
    timefmt='%Y-%m-%d %H:%M:%S',
    until=None,
    ids=None,
    **query,
)
Docstring:
List runs from catalog.

This function provides a thin interface to the highly-reconfigurable
``ListRuns()`` class in this package.

PARAMETERS

cat
    *object* :
    Instance of databroker v1 or v2 catalog.
keys
    *str* or *[str]* or None:
    Include these additional keys from the start document.
    (default: ``None`` means ``"scan_id time plan_name detectors"``)
missing
    *str*:
    Test to report when a value is not available.
    (default: ``""``)
ids
    *[int]* or *[str]*:
    List of ``uid`` or ``scan_id`` value(s).
    Can mix different kinds in the same list.
    Also can specify offsets (e.g., ``-1``).
    According to the rules for ``databroker`` catalogs,
    a string is a ``uid`` (partial representation

### Search within a range of dates

To find runs that started since a particular date, use `listruns(since="yyyy-mm-dd hh:mm")` where `yyyy-mm-dd hh:mm` is a suggestion.  You only need to supply the parts that matter, so "2:00" would find all runs that started after 2 AM today.

Here, we look for runs **since** the beginning of April 2021.  (Because the date is
incomplete, it is implied that the full specification is `2021-04-01
00:00:00.0000000`.)

In [8]:
listruns(since="2021-04")

/home/prjemian/.conda/envs/bluesky_2022_3/lib/python3.9/site-packages/databroker/queries.py:89: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  timezone = lz.zone


catalog: class_2021_03
   scan_id                 time plan_name                   detectors
0      192  2021-05-19 15:22:03  rel_scan  [scaler1, adsimdet, zaxis]
1      191  2021-05-19 15:19:27  rel_scan         [scaler1, adsimdet]
2      190  2021-05-19 15:18:59  rel_scan                   [scaler1]
3      189  2021-05-19 15:18:28  rel_scan      [scaler1, temperature]
4      188  2021-05-19 15:14:56  rel_scan                   [scaler1]
5      187  2021-05-19 15:13:47  rel_scan                   [scaler1]
6       33  2021-04-07 15:55:09      scan                     [noisy]


Likewise, find runs that started before a particular date and time using `listruns(until="yyyy-mm-dd hh:mm")`

Here, the specification will include any date **until** (before) the beginning of May 2021.

In [9]:
listruns(until="2021-05")

/home/prjemian/.conda/envs/bluesky_2022_3/lib/python3.9/site-packages/databroker/queries.py:89: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  timezone = lz.zone


catalog: class_2021_03
    scan_id                 time plan_name               detectors
0        33  2021-04-07 15:55:09      scan                 [noisy]
1        33  2021-03-17 00:32:55     count              [adsimdet]
2        32  2021-03-17 00:31:44     count           [temperature]
3        31  2021-03-17 00:31:24     count           [temperature]
4        30  2021-03-17 00:31:23     count           [temperature]
5        29  2021-03-17 00:31:23     count           [temperature]
6        28  2021-03-17 00:30:33  rel_scan                 [noisy]
7        27  2021-03-17 00:30:27  rel_scan                 [noisy]
8        26  2021-03-17 00:30:21  rel_scan                 [noisy]
9        25  2021-03-17 00:30:04  rel_scan                 [noisy]
10       24  2021-03-17 00:29:57  rel_scan                 [noisy]
11       23  2021-03-17 00:29:40  rel_scan                 [noisy]
12       22  2021-03-17 00:27:55     count        [scaler1, noisy]
13       21  2021-03-17 00:27:23     co

You can combine them:

In [10]:
listruns(since="2021-04", until="2021-05")

/home/prjemian/.conda/envs/bluesky_2022_3/lib/python3.9/site-packages/databroker/queries.py:89: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  timezone = lz.zone


catalog: class_2021_03
   scan_id                 time plan_name detectors
0       33  2021-04-07 15:55:09      scan   [noisy]


### Search metadata keys

Bluesky stores a run's metadata in the `start` document.  Any of the terms may
be searched by adding keyword arguments to `listruns()`, the value is the
content to match.

Since only a few metadata keys are *standard* (such as `time` and `uid`), you should be prepared for the possibility that any particular key may not be found.

Let's take a look at the metadata of the most recent run in the catalog:

In [11]:
run = cat[-1]
run.metadata

{'start': Start({'beamline_id': 'APS_Python_training_2021',
 'detectors': ['scaler1', 'adsimdet', 'zaxis'],
 'hints': {'dimensions': [[['zaxis_h'], 'primary']]},
 'instrument_name': 'class_2021_03',
 'login_id': 'prjemian@zap',
 'motors': ['zaxis_h'],
 'notebook': 'UB_autosave',
 'num_intervals': 7,
 'num_points': 8,
 'objective': 'Demonstrate UB matrix save & restore',
 'pid': 3712584,
 'plan_args': {'args': ["PseudoSingle(prefix='', name='zaxis_h', "
                        "parent='zaxis', settle_time=0.0, timeout=None, "
                        "egu='', limits=(0, 0), source='computed', "
                        "read_attrs=['readback', 'setpoint'], "
                        'configuration_attrs=[], idx=0)',
                        -0.1,
                        0.1],
               'detectors': ["ScalerCH(prefix='gp:scaler1', name='scaler1', "
                             "read_attrs=['channels', 'channels.chan01', "
                             "'channels.chan01.s', 'channels.chan

A plan's name is stored in the run's metadata as the `plan_name` key.  (There is
no guarantee that a run will have this metadata key.)  To list runs measured
with the `count` plan, use `listruns(plan_name="count")`.

In [12]:
listruns(plan_name="count")

/home/prjemian/.conda/envs/bluesky_2022_3/lib/python3.9/site-packages/databroker/queries.py:89: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  timezone = lz.zone


catalog: class_2021_03
    scan_id                 time plan_name         detectors
0        33  2021-03-17 00:32:55     count        [adsimdet]
1        32  2021-03-17 00:31:44     count     [temperature]
2        31  2021-03-17 00:31:24     count     [temperature]
3        30  2021-03-17 00:31:23     count     [temperature]
4        29  2021-03-17 00:31:23     count     [temperature]
5        22  2021-03-17 00:27:55     count  [scaler1, noisy]
6        21  2021-03-17 00:27:23     count         [scaler1]
7        20  2021-03-17 00:27:22     count         [scaler1]
8         5  2021-03-15 11:54:56     count     [temperature]
9         4  2021-03-15 11:49:42     count     [temperature]
10        3  2021-03-15 11:46:21     count     [temperature]
11        2  2021-03-15 11:44:21     count     [temperature]
12        1  2021-03-15 00:52:29     count     [temperature]


You can combine a search with more than one metadata key, such as *any `count` run `#20`*.

In [13]:
listruns(plan_name="count", scan_id=20)

/home/prjemian/.conda/envs/bluesky_2022_3/lib/python3.9/site-packages/databroker/queries.py:89: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  timezone = lz.zone


catalog: class_2021_03
   scan_id                 time plan_name  detectors
0       20  2021-03-17 00:27:22     count  [scaler1]


### Search metadata keys using MongoDB Query

Searching for a specific `scan_id` is awkward.  Since we know that `scan_id` is
stored as a number, we can apply range limits (such as `30 <= scan_id < 100`).
That type of search requires the syntax from [MongoDB
Query](https://www.mongodb.com/docs/manual/reference/operator/query/).

A Query is built as a Python dictionary where the comparison operators are the
keys and the comparison values are the corresponding values.  These are the
terms we need for this query:

operator | MongoDB Query
--- | ---
`>=` | `"$gte"`
`<` | `"$lt"`

The full Query is `scan_id={"$gte": 30, "$lt": 100}`.

Here, search for `scan_id` matching that Query and a `count` plan:

In [14]:
listruns(scan_id={"$gte": 30, "$lt": 100}, plan_name="count")

/home/prjemian/.conda/envs/bluesky_2022_3/lib/python3.9/site-packages/databroker/queries.py:89: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  timezone = lz.zone


catalog: class_2021_03
   scan_id                 time plan_name      detectors
0       33  2021-03-17 00:32:55     count     [adsimdet]
1       32  2021-03-17 00:31:44     count  [temperature]
2       31  2021-03-17 00:31:24     count  [temperature]
3       30  2021-03-17 00:31:23     count  [temperature]


## Filtered catalog using MongoDB Query

As an alternative to `listruns()`, you can create your own catalogs by applying MongoDB Query searches to an existing catalog.

Let's start with a larger initial catalog to demonstrate.

In [15]:
cat = databroker.catalog["training"].v2
print(f"There are {len(cat)} runs in the '{cat.name}' catalog.")

There are 8116 runs in the 'training' catalog.


**Q**: How many runs use the `count` plan?

To answer this, we must know how to apply the MongoDB Query to the catalog.  The catalog has a `.search(query)` method, where the `query` term is the dictionary, similar to how it was used above.  Instead of a keyword, though, the metadata key is ***in*** the dictionary.

In [16]:
query = {"plan_name":{"$eq": "count"}}
print(f"{query=}")
filtered_cat = cat.search(query)
print(f"There are {len(filtered_cat)} runs collected by the 'count()' plan.")

query={'plan_name': {'$eq': 'count'}}
There are 295 runs collected by the 'count()' plan.


Searches can get more detailed.  Find all `count` runs with `scan_id` between 70
and 80. (This catalog has duplicates in this range.)  Display the filtered
catalog using `listruns()`

In [17]:
filtered_cat = cat.search({"plan_name":{"$eq": "count"}, "scan_id": {"$gt": 70, "$lt": 80}})
print(f"{len(filtered_cat)=}")
listruns(filtered_cat)

len(filtered_cat)=18


/home/prjemian/.conda/envs/bluesky_2022_3/lib/python3.9/site-packages/databroker/queries.py:89: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  timezone = lz.zone


catalog: search results
    scan_id                 time plan_name                    detectors
0        79  2022-05-28 13:09:24     count                     [simdet]
1        78  2022-05-28 13:06:23     count                   [adsimdet]
2        77  2022-05-27 12:37:54     count                   [adsimdet]
3        76  2022-05-27 12:28:50     count                   [adsimdet]
4        75  2022-05-27 12:27:19     count                     [simdet]
5        74  2022-05-26 17:29:10     count                     [simdet]
6        73  2022-05-26 17:25:14     count                     [simdet]
7        72  2022-05-26 17:24:33     count                     [simdet]
8        71  2022-05-26 17:23:51     count                     [simdet]
9        79  2021-04-15 09:00:24     count  [scaler1, count_difference]
10       78  2021-04-15 09:00:22     count  [scaler1, count_difference]
11       77  2021-04-15 08:59:20     count  [scaler1, count_difference]
12       76  2021-04-15 08:59:15     cou

-----------

## Other searches: 

<details>
<summary>TODO:</summary>

Some of these seaches may need additional Python code to complete.  Others may
be expedited by additional MongoDB Query constructs.

* What version of `bluesky` was used 6 months ago?
* When was `apstools` version 1.2 - 1.4 used?
* Find all runs with sample "xyz" measured with detector `adsimdet`.  List the most recent ones.
  * Do we have any APS catalogs with sample name as metadata?
  * similar: `listruns(detectors={"$in": ["adsimdet"]}, plan_name="count")`
* Restrict that list to runs in APS cycle `2021-3` (if no APS cycle info, the last 4 months of 2021).
* Find runs where "Joe User" appears somewhere in the metadata.
* Find runs where "silver behenate" appears somewhere in the metadata.
* Find runs that failed for some reason.  Is there any indication why?
* Plot centroid and width of all successful scans of specific y vs. x with more than one data point for the last week.
* List `scan` (or `rel_scan`) including these keys
   -  plan
   -  dets
   -  md
   -  start position
   -  end position
   -  number of points
- What about these?
   -  using start or stop document metadata
   -  user
   -  sample
   -  Proposal or ESAF ID
   -  fuzzy or misspelled terms
   -  combination searches

<details>